In [1]:
!pip install -Uqq gradio

In [2]:
!pip install -Uqq scikit-image

In [3]:
from fastai.vision.all import *

In [4]:
import skimage

In [5]:
# Import data through untar_data function which takes in URLs."dataset name"
path = untar_data(URLs.PETS)

In [6]:
# Instantiate data loaders object dls. this object contains all the images we will train model on
dls = ImageDataLoaders.from_name_re(
    path,
    get_image_files(path/'images'),
    pat='(.+)_\d+.jpg',
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224, min_scale=0.75)
)

In [7]:
path

Path('/root/.fastai/data/oxford-iiit-pet')

In [8]:
# Instantiate another object learn from the vision learner class
learn = vision_learner(dls, models.resnet50, metrics=accuracy)
learn.fine_tune(1)

/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/opt/conda/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,accuracy,time
0,0.979810,0.309610,0.910014,01:35


epoch,train_loss,valid_loss,accuracy,time
0,0.451637,0.237943,0.923545,01:40


In [9]:
learn.path = Path('.')


In [10]:
learn.export(fname='/kaggle/working/breedclassifiermodel.pkl',pickle_protocol=2)

In [11]:
learn = load_learner('breedclassifiermodel.pkl')

In [12]:
#Defining a prediction function for the model
labels = learn.dls.vocab
def predict(img):
    img = PILImage.create(img)
    pred,pred_idx,probs = learn.predict(img)
    return {labels[i]: float(probs[i]) for i in range(len(labels))}

In [13]:
#importing gradio and creating the interface
import gradio as gr
gr.Interface(fn=predict, inputs=gr.Image(shape=(512, 512)), outputs=gr.Label(num_top_classes=3)).launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://00f03b23c1be8e8a3a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


All of these features are arguments for the instantiation of the Interface class.

In [14]:
title = "Pet Breed Classifier"

description = "A pet breed classifier trained on the Oxford Pets dataset with fastai. Created as a demo for Gradio and HuggingFace Spaces."

article = "<p style='text-align: center'><a href='https://tmabraham.github.io/blog/gradio_hf_spaces_tutorial' target='_blank'>Blog post</a></p>"

examples = [Path('/root/.fastai/data/oxford-iiit-pet/images/Abyssinian_1.jpg')]

Another interesting feature that Gradio has is the ability for interpretation so that users can understand what parts of the input are responsible for the output. We'll use the default interpretation function provided by Gradio

In [15]:
# Note that the default interpretation function needs scikit-image to be installed.
interpretation ='default'

Gradio also supports serving of inference requests with a queue. This can be helpful when your app receives a significant amount of traffic. We'll enable a queue here:

In [16]:
enable_queue=True

In [17]:
#rerunning the Interface method with the additional arguements
gr.Interface(
    fn=predict,
    inputs=gr.Image(shape=(512, 512)),
    outputs=gr.Label(num_top_classes=3),
    title=title,
    description=description,
    article=article,
    examples=examples,
    interpretation=interpretation,
    ).launch(share=True, enable_queue=enable_queue)


Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://610eea6a6b02527b6d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
